In [13]:
import os
import json
import pandas as pd
import traceback

In [14]:
from langchain_openai import ChatOpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv("/Users/ss11/mcqgen/src/.env")

False

In [17]:
KEY=os.getenv("OPENAI_API_KEY")

In [22]:
print("KEY:", KEY)

KEY: None


In [12]:
llm=ChatOpenAI(api_key=KEY, model="gpt-3.5-turbo", temperature=0.5)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)